### Используемые библиотеки

In [ ]:
import pandas as pd
from skopt import BayesSearchCV
from catboost import CatBoostClassifier
from skopt.space import Categorical, Real
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [41]:
# считываем обработанные данные
df = pd.read_csv("data.csv")
df_y = df[['IC50, mM', 'CC50, mM', 'SI']].copy()
df_x = df.drop(columns=['IC50, mM', 'CC50, mM', 'SI'])

# добавляем новый признак согласно заданию
median = df_y['IC50, mM'].median()
df_y['bool'] = (df_y['IC50, mM'] > median).astype(bool)


### Классификация IC50 (превышение медианного значения выборки)

In [42]:
# датасет для сравнения метрик по каждой из модели
metrics = pd.DataFrame(columns=["Accuracy",
                                "Precision",
                                "Recall",
                                "F1-score",
                                "ROC AUC"]
                                )

In [43]:
# избавимся от лишних признаков, выделим целевую переменную и добавим дополнительный признак
y_data = df_y['bool']
x_data = df_x
x_data['CC50, mM'] = df_y['CC50, mM']

# сократим количество признаков до 50 с помощью Mutual Info
selector = SelectKBest(score_func=mutual_info_regression, k=50)
select_x_data = selector.fit_transform(x_data, y_data)
selected_features = x_data.columns[selector.get_support()]
x_data = pd.DataFrame(select_x_data, columns=selected_features)

# разделим данные на тестовыые и тренировочные
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
print(f'Тренеровочные данные: {X_train.shape}, {y_train.shape}')
print(f'Tестовые данные: {X_test.shape}, {y_test.shape}')

# стандартизируем данные train
scaler = StandardScaler()
tmp_data = scaler.fit_transform(X_train)
scaled_X_train = pd.DataFrame(tmp_data, columns=x_data.columns)

# стандартизируем данные test
tmp_data = scaler.transform(X_test)
scaled_X_test = pd.DataFrame(tmp_data, columns=x_data.columns)

Тренеровочные данные: (772, 50), (772,)
Tестовые данные: (194, 50), (194,)


#### RandomForestClassifier

In [44]:
# определим пространство гиперпараметров для модели RandomForestClassifier
param_space_rfr = {
    'n_estimators': (50, 500),
    'max_depth': (1, 50),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 10),
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True, False])
}

# создадим модель
model = RandomForestClassifier(random_state=42)

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_rfr,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
rfc_model = bayes_search.best_estimator_


ROC AUC = 0.8266

Лучшие параметры модели:
bootstrap = True
max_depth = 50
max_features = sqrt
min_samples_leaf = 10
min_samples_split = 20
n_estimators = 500


In [45]:
# расcчитаем y
y_pred = rfc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[0] = res

Accuracy: 0.8299
Precision: 0.8571
Recall: 0.75
F1-score: 0.8


#### GradientBoostingClassifier

In [46]:
# определим пространство гиперпараметров для модели GradientBoostingClassifier
param_space_gbr = {
    'n_estimators': (10, 200),
    'max_depth': (1, 20),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 7),
}

# создадим модель
model = GradientBoostingClassifier(random_state=42)

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_gbr,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
gbc_model = bayes_search.best_estimator_

ROC AUC = 0.8407

Лучшие параметры модели:
max_depth = 4
min_samples_leaf = 4
min_samples_split = 6
n_estimators = 200


In [47]:
# расcчитаем y
y_pred = gbc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[1] = res

Accuracy: 0.8093
Precision: 0.8493
Recall: 0.7045
F1-score: 0.7702


#### CatBoostClassifier

In [48]:
# определим пространство гиперпараметров для модели CatBoostClassifier
param_space_cbc = {
    'iterations': (100, 1000),
    'depth': (4, 10),
    'grow_policy': Categorical(['SymmetricTree', 'Depthwise', 'Lossguide']),
    'loss_function': Categorical(['Logloss', 'CrossEntropy']),
    'l2_leaf_reg': Real(1, 10),
    'min_data_in_leaf': (1, 50),
    'one_hot_max_size': (2, 10)
}

# создадим модель
model = CatBoostClassifier()

# оптимизируем с помощью Байесовского поиска
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space_cbc,
    n_iter=20,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
)

bayes_search.fit(scaled_X_train, y_train)

# выведем показатели
roc_auc = round(bayes_search.best_score_, 4)
print(f'ROC AUC = {roc_auc}\n')
print('Лучшие параметры модели:')
for k, v in bayes_search.best_params_.items():
    print(f"{k} = {v}")

# сохраним модель с наилучшими параметрами
cbc_model = bayes_search.best_estimator_

0:	learn: 0.6793996	total: 7.3ms	remaining: 6.85s
1:	learn: 0.6646228	total: 12.1ms	remaining: 5.66s
2:	learn: 0.6506014	total: 26.2ms	remaining: 8.18s
0:	learn: 0.6793398	total: 23.9ms	remaining: 22.4s
3:	learn: 0.6380374	total: 46.5ms	remaining: 10.9s
0:	learn: 0.6797819	total: 26ms	remaining: 24.5s
0:	learn: 0.6802351	total: 25.7ms	remaining: 24.1s
1:	learn: 0.6699314	total: 38.3ms	remaining: 18s
1:	learn: 0.6645288	total: 44ms	remaining: 20.6s
0:	learn: 0.6791948	total: 24.1ms	remaining: 22.6s
4:	learn: 0.6241417	total: 65.7ms	remaining: 12.3s
1:	learn: 0.6676731	total: 43.9ms	remaining: 20.6s
1:	learn: 0.6681197	total: 40.2ms	remaining: 18.9s
2:	learn: 0.6502396	total: 62.6ms	remaining: 19.5s
2:	learn: 0.6581507	total: 60.9ms	remaining: 19s
5:	learn: 0.6116088	total: 87.3ms	remaining: 13.6s
2:	learn: 0.6544867	total: 65.7ms	remaining: 20.5s
2:	learn: 0.6560594	total: 60.6ms	remaining: 18.9s
3:	learn: 0.6359925	total: 83.3ms	remaining: 19.5s
3:	learn: 0.6445272	total: 79.9ms	remain

In [49]:
# расcчитаем y
y_pred = cbc_model.predict(scaled_X_test)

# оценим метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {round(accuracy, 4)}")
print(f"Precision: {round(precision, 4)}")  
print(f"Recall: {round(recall, 4)}")        
print(f"F1-score: {round(f1, 4)}")

# сохраним данные в таблицу
res = {"Accuracy": round(accuracy, 4),
       "Precision": round(precision, 4),
       "Recall": round(recall, 4),
       "F1-score": round(f1, 4),
       "ROC AUC": round(roc_auc, 4)}

metrics.loc[2] = res

Accuracy: 0.8505
Precision: 0.8554
Recall: 0.8068
F1-score: 0.8304


In [50]:
metrics.index = ['RandomForestClassifier', 'GradientBoostingClassifier', 'CatBoostClassifier']
metrics.head()

,Accuracy,Precision,Recall,F1-score,ROC AUC
RandomForestClassifier,0.8299,0.8571,0.7500,0.8000,0.8266
GradientBoostingClassifier,0.8093,0.8493,0.7045,0.7702,0.8407
CatBoostClassifier,0.8505,0.8554,0.8068,0.8304,0.8430


#### Вывод
Было выполнено преобразование признака IC50, mM по бинарному критерию. Критерий оценки состоял из сравнения медианного значения признака IC50, mM с текущим значением данных.

Таким образом данные превосзодившие медианное значение IC50, mM получали значение True, равные или меньшие получали значение False.

Для булевой целевой переменной IC50 были дообработанны данные:
* Добавлен дополнительный признак CC50, mM в данные для обучения
* Сокращено количество признаков до 50
* Данные разделены на тестовыые и тренировочные
* Проведена стандартизация тренировочных данных и на её основе стандартизированы тестовые данные

Затем были подобраны параметры и построены три модели:

* RandomForestClassifier c параметрами:
    - bootstrap = True
    - max_depth = 50
    - max_features = sqrt
    - min_samples_leaf = 10
    - min_samples_split = 20
    - n_estimators = 500

* GradientBoostingClassifier с параметрами:
    - max_depth = 4
    - min_samples_leaf = 4
    - min_samples_split = 6
    - n_estimators = 200

* CatBoostClassifier с параметрами:
    - depth = 5
    - grow_policy = Depthwise
    - iterations = 103
    - l2_leaf_reg = 1.0131357873772266
    - loss_function = Logloss
    - min_data_in_leaf = 15
    - one_hot_max_size = 2

В качестве метрик для оценки работы модели использовались:
* Accuracy
* Precision
* Recall
* F1-score
* ROC AUC

Наилучшие показатели метрик и при обучение и при валидации на тестовых данных получились у модели CatBoostClassifier.